<a href="https://colab.research.google.com/github/honginhwa/pytorch_study/blob/master/dacon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms

import os
import cv2
import copy
import time
from random import *
from collections import defaultdict

from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/데이콘 컴퓨터비전 학습 경진대회/data/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/데이콘 컴퓨터비전 학습 경진대회/data/test.csv')
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/데이콘 컴퓨터비전 학습 경진대회/data/submission.csv')


In [3]:
train.head()

,id,digit,letter,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783
0,1,5,L,1,1,1,4,3,0,0,4,4,3,0,4,3,3,3,4,4,0,0,1,1,3,4,0,4,2,0,4,0,1,3,1,0,4,1,1,3,...,4,3,4,1,3,0,0,1,3,3,3,0,3,2,2,1,0,1,0,0,3,0,0,4,2,0,3,4,1,1,2,1,0,1,2,4,4,4,3,4
1,2,0,B,0,4,0,0,4,1,1,1,4,2,0,3,4,0,0,2,3,4,0,3,4,3,0,2,2,1,4,2,3,3,4,1,2,4,2,0,3,...,4,2,3,0,0,0,0,4,3,2,2,4,2,1,1,1,3,3,1,2,4,4,4,2,2,4,4,0,4,2,0,3,0,1,4,1,4,2,1,2
2,3,4,L,1,1,2,2,1,1,1,0,2,1,3,2,2,2,4,1,1,4,1,0,1,3,4,2,2,2,4,1,1,2,0,3,0,2,3,4,0,...,3,0,4,0,3,0,2,0,1,4,2,3,4,4,4,0,2,0,4,4,1,3,0,3,2,0,2,3,0,2,3,3,3,0,2,0,3,0,2,2
3,4,9,D,1,2,0,2,0,4,0,3,4,3,1,0,3,2,2,0,3,4,1,0,4,1,2,2,3,2,2,0,2,0,3,0,3,2,4,0,0,...,0,3,0,1,4,1,3,1,2,1,1,1,2,2,2,4,3,4,3,0,4,1,2,4,1,4,0,1,0,4,3,3,2,0,1,4,0,0,1,1
4,5,6,A,3,0,2,4,0,3,0,4,2,4,2,1,4,1,1,4,4,0,2,3,4,4,3,3,3,3,4,1,0,3,0,3,0,0,0,1,1,...,2,1,3,2,1,4,2,3,2,2,1,0,4,2,2,1,2,1,0,3,2,2,2,2,1,4,2,1,2,1,4,4,3,2,1,3,4,3,1,2


In [4]:
x_train = np.concatenate(
    [
     pd.get_dummies(train['letter']).values.reshape(-1,1,26),                     #letter 레이블 만들기 
     (train[[str(i) for i in range(784)]]/255) .values.reshape(-1,1,784)],axis=2  #그림 값들  
     )

y_train = train['digit'].values


In [5]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=123)

In [6]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(1638, 1, 810)
(410, 1, 810)
(1638,)
(410,)


In [7]:
x_train = torch.Tensor(x_train)
x_valid = torch.Tensor(x_valid)
y_train = torch.LongTensor(y_train)
y_valid = torch.LongTensor(y_valid)

##### concat letter + image + digit

### sequentialsampler  
- sampler는 데이터에 인덱스를 불러오는 방법입니다
그래서 시퀀셜샘플러는 그냥 차례데로 불러오는 샘플러입니다

### DataLoader
- 데이터셋을 인자로 받아 뽑아넴



In [8]:
train_data=TensorDataset(
    x_train[:,:,:26], # letter
    x_train[:,:,26:].reshape(-1,1,28,28), #image
    y_train) # digit

train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

valid_data = TensorDataset(
    x_valid[:,:,:26],# concat letter
    x_valid[:,:,26:].reshape(-1,1,28,28), # 행렬, 행렬reshape
    y_valid) # digit

valid_sampler = SequentialSampler(valid_data) # sequentialsampler  같은 순서
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler,batch_size=32)


#### Output Size = (W + 2xP - Dx(K-1) -1) / S + 1

- W: input_volume_size

- K: kernel_size

- D: Dilation

- P: padding_size

- S: strides

In [9]:
class customCNN(nn.Module):
  def __init__(self):
    super().__init__()
    #letter의 conv
    self.conv1 = nn.Sequential(
        nn.Conv1d(1,16,3,padding=1),nn.ReLU(),    
        nn.Conv1d(16,64,4,padding=1),nn.ReLU(),    
        nn.Conv1d(64,128,5,padding=2),nn.ReLU(),
        nn.Conv1d(128,64,4,padding=1),nn.ReLU(),
        nn.Conv1d(64,16,3),nn.ReLU()
    )

    #image의 conv
    self.conv2 = nn.Sequential(
        nn.Conv2d(1,32,3,padding=1),nn.ReLU(),        # 28x28x32
        nn.Conv2d(32,128,5,padding=2),nn.ReLU(),      # 28x28x128
        nn.Conv2d(128,256,7,padding=3),nn.ReLU(),     # 26x26x512
        nn.Conv2d(256,512,9,padding=3),nn.ReLU(),     # 24x24x512
        nn.Conv2d(512,256,9,padding=3),nn.ReLU(),     # 24x24x256
        nn.Conv2d(256,128,7,padding=3),nn.ReLU(),     # 24x24x128
        nn.Conv2d(128,64,7,padding=3),nn.ReLU(),      # 24x24x64
        nn.Conv2d(64,32,5,padding=3),nn.ReLU(),       # 26x26x32
    )

    self.out = nn.Sequential(
        nn.Linear(22016,128), nn.ReLU(), #letter + image
        nn.Linear(128,32), nn.ReLU(),
        nn.Linear(32,10)
    )
    self.loss = nn.CrossEntropyLoss()

    def forward(self, x1, x2):
      out = self._inference(x1,x2)
      if label is not False:
        loss = self.loss(out, label)
        return (out, loss)

      return out

    def _inference(self, x1, x2):
      bsz = x1.size(0)
      x1 = self.conv1(x1)
      x2 = self.conv2(x2)

      x1 = x1.view(bsz,-1)
      x2 = x2.view(bsz,-1)

      x = torch.cat([x1,x2], dim=1)
      out = torch.nn.functional.softmax(self.out(x),dim=1)

      return out

In [10]:
model = customCNN()
model.cuda()

customCNN(
  (conv1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(16, 64, kernel_size=(4,), stride=(1,), padding=(1,))
    (3): ReLU()
    (4): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,))
    (5): ReLU()
    (6): Conv1d(128, 64, kernel_size=(4,), stride=(1,), padding=(1,))
    (7): ReLU()
    (8): Conv1d(64, 16, kernel_size=(3,), stride=(1,))
    (9): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): ReLU()
    (4): Conv2d(128, 256, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (5): ReLU()
    (6): Conv2d(256, 512, kernel_size=(9, 9), stride=(1, 1), padding=(3, 3))
    (7): ReLU()
    (8): Conv2d(512, 256, kernel_size=(9, 9), stride=(1, 1), padding=(3, 3))
    (9): ReLU()
    (10): Conv2d(256, 128, kernel_size=(7, 7), stride=(1, 1), padd

In [11]:
test_letter = x_train[:32, : , :26].cuda()
test_image = x_train[:32, : , 26:].reshape(-1,1,28,28).cuda()

In [12]:
print(test_letter.shape)
print(test_image.shape)

torch.Size([32, 1, 26])
torch.Size([32, 1, 28, 28])


In [14]:

model(test_letter, test_image)

NotImplementedError: ignored

In [ ]:
from torch.optim import Adam

optimizer = Adam(
    model.parameters().
    lr=2e-5,
    eps=1e-8
)